In [ ]:
!unzip -o dlgo.zip

Archive:  dlgo.zip
   creating: dlgo/
   creating: dlgo/agent/
  inflating: dlgo/agent/base.py      
  inflating: dlgo/agent/helpers.py   
  inflating: dlgo/agent/naive.py     
  inflating: dlgo/agent/predict.py   
 extracting: dlgo/agent/__init__.py  
   creating: dlgo/agent/__pycache__/
  inflating: dlgo/agent/__pycache__/base.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/__init__.cpython-310.pyc  
   creating: dlgo/data/
  inflating: dlgo/data/generator.py  
  inflating: dlgo/data/index_processor.py  
  inflating: dlgo/data/parallel_processor.py  
  inflating: dlgo/data/processor.py  
  inflating: dlgo/data/sampling.py   
  inflating: dlgo/data/__init__.py   
   creating: dlgo/data/__pycache__/
  inflating: dlgo/data/__pycache__/index_processor.cpython-310.pyc  
  inflating: dlgo/data/__pycache

Загрузка признаков и меток с помощью обработчика данных

In [ ]:
import h5py

from keras.models import Sequential
from keras.layers import Dense

from dlgo.agent.predict import DeepLearningAgent, load_prediction_agent
from dlgo.data.parallel_processor import GoDataProcessor
#from dlgo.data.processor import GoDataProcessor
from dlgo.encoders.sevenplane import SevenPlaneEncoder
from dlgo.networks import large

from dlgo.httpfrontend import get_web_app

In [ ]:
go_board_rows, go_board_cols = 19, 19
nb_classes = go_board_rows * go_board_cols
encoder = SevenPlaneEncoder((go_board_rows, go_board_cols))
processor = GoDataProcessor(encoder=encoder.name())

In [ ]:
%%time

#X, y = processor.load_go_data(num_samples=100)
X, y = processor.load_go_data_npy()

CPU times: user 1.57 ms, sys: 50 ms, total: 51.6 ms
Wall time: 51.8 ms


Создание и запуск модели для предсказания ходов с использованием оптимизатора Adadelta

In [ ]:
%%time

input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
model = Sequential()
network_layers = large.layers(input_shape)
for layer in network_layers:
    model.add(layer)
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
88/88 [==============================] - 9s 30ms/step - loss: 5.8890 - accuracy: 0.0029
Epoch 2/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8890 - accuracy: 0.0033
Epoch 3/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8889 - accuracy: 0.0036
Epoch 4/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8889 - accuracy: 0.0037
Epoch 5/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8888 - accuracy: 0.0038
Epoch 6/20
88/88 [==============================] - 3s 31ms/step - loss: 5.8888 - accuracy: 0.0043
Epoch 7/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8888 - accuracy: 0.0042
Epoch 8/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8887 - accuracy: 0.0044
Epoch 9/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8887 - accuracy: 0.0044
Epoch 10/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8887 - accuracy: 0.0044
Epoch 11/

Создание и сохранение агента DeepLearningAgent

In [ ]:
deep_learning_bot = DeepLearningAgent(model, encoder)
h5file = h5py.File("./agents/deep_bot.h5", "w")
deep_learning_bot.serialize(h5file)

>>>> /tmp/tmp-kerasmodelazr8xlwo.h5


In [ ]:
#import h5py
#from dlgo import kerasutil
#deep_learning_bot = DeepLearningAgent(model, encoder)
#h5file = h5py.File("./deep_bot.h5", "w")
#h5file.create_group('encoder')
#h5file['encoder'].attrs['name'] = deep_learning_bot.encoder.name()
#h5file['encoder'].attrs['board_width'] = deep_learning_bot.encoder.board_width
#h5file['encoder'].attrs['board_height'] = deep_learning_bot.encoder.board_height
#h5file.create_group('model')
#kerasutil.save_model_to_hdf5_group(deep_learning_bot.model, h5file['model'])

Загрузка бота обратно в память...

In [ ]:
model_file = h5py.File("./agents/deep_bot.h5", "r")
bot_from_file = load_prediction_agent(model_file)

...и его подключение к веб-приложению

In [ ]:
#web_app = get_web_app({'predict': bot_from_file})
#web_app.run()